<a href="https://colab.research.google.com/github/linyuehzzz/hedetniemi_distance/blob/master/dijkstra_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Dijkstra distance**
Dijkstra's shortest path algorithm (codes from [Ningchuan Xiao](https://github.com/gisalgs/networks/blob/master/dijkstra.py))   
Yue Lin (lin.3326 at osu.edu)  
Created: 6/20/2020

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

#### **Install packages** 

In [1]:
!pip install timeout-decorator

  Created wheel for timeout-decorator: filename=timeout_decorator-0.4.1-cp36-none-any.whl size=5021 sha256=590a27ae4770fb6f8bbb519ffaaa501bbf8f6f88f8dbafa9f66194c86b99deb7
  Stored in directory: /root/.cache/pip/wheels/f1/e6/ea/7387e3629cb46ba65140141f972745b823f4486c6fe884ccb8
Successfully built timeout-decorator


#### **Read graph data** 

##### Data from the original article

In [2]:
## [node i, node j, distance between node i and j]
## using data from example 1: San Francisco Bay Area Graph of Time-Distances (in minutes)
nodes = 9
source = 1
destination = 7

data = [[1, 2, 30], [1, 4, 30], [1, 9, 40],
        [2, 3, 25], [2, 4, 40], [3, 4, 50],
        [4, 5, 30], [4, 6, 20], [5, 7, 25],
        [6, 7, 20], [6, 9, 20], [7, 8, 25],
        [8, 9, 20]]

##### Read random graph

In [ ]:
%cd '/content/gdrive/My Drive/Colab Notebooks/hedetniemi_matrix_sum'

## Number of nodes (100/1,000/10,000/100,000/1,000,000)
nodes = 1000
print('Nodes: ', nodes)
## Total degree
degree = 3
print('Degree: ', degree)

data = []
with open('graph_n' + str(nodes) + '_d' + str(degree) + '.txt', 'r') as f:
  lines = f.read().splitlines()
  for line in lines:
    l = line.split()
    item = [int(l[0]), int(l[1]), float(l[2])]
    data.append(item)

print(data[0])

/content/gdrive/My Drive/Colab Notebooks/hedetniemi_matrix_sum
Nodes:  1000
Degree:  3
[609, 621, 18.019071417527243]


#### **Implementation 1: list** 

##### Construct distance matrix

In [ ]:
from timeit import default_timer
import timeout_decorator

@timeout_decorator.timeout(100)
def distance_matrix(graph, n):
  ## calculate distance matrix
  INF = float('inf')
  dist_mtx = [[INF] * n for i in range(n)]
  for g in graph:
    i = g[0] - 1
    j = g[1] - 1
    d = g[2]
    dist_mtx[i][j] = d
    dist_mtx[j][i] = d

  ## set diagonal to 0
  for i in range(n):
    dist_mtx[i][i] = 0.0
 
  return dist_mtx


## print time costs
try:
  start = default_timer()
  dist_mtx = distance_matrix(data, nodes)
  stop = default_timer()
  print('Time: ', stop - start)
except:
  print('Time: inf')

Time:  0.00017785399995773332


##### Calculate Dijkstra distance

In [ ]:
from timeit import default_timer
import timeout_decorator

INF = float('inf')

@timeout_decorator.timeout(100)
def dijkstra(source, distmatrix, n):
  dist = [INF if i!=source else 0 for i in range(n)]
  Q = list(range(n))
  while len(Q)>0:
    u = get_remove_min(Q, dist)
    U = get_neighbor(u, distmatrix, n)
    for v in U:
      newd = dist[u] + distmatrix[u][v]
      if newd < dist[v]:
        dist[v] = newd
  return dist


def get_remove_min(Q, dist):
  dmin = INF
  imin = -1
  for i in Q:
    if dist[i] < dmin:
      dmin = dist[i]
      imin = i
  Q.remove(imin)
  return imin


def get_neighbor(u, d, n):
  neighbors = [i for i in range(n) if d[i][u]!=INF and i!=u]
  return neighbors


def shortest_path(source, destination, distmatrix, n):
  dist = dijkstra(source, distmatrix, n)
  return dist[destination]

## print time costs
try:
  start = default_timer()
  dist = shortest_path(source, destination, dist_mtx, nodes)
  stop = default_timer()
  print('Time: ', stop - start)
  
except:
  print('Time: inf')
  raise

Time:  0.00010139500000150292


#### **Implementation 2: numpy** 

##### Construct distance matrix

In [3]:
from timeit import default_timer
import numpy as np
import timeout_decorator

@timeout_decorator.timeout(100)
def distance_matrix(graph, n):
  ## calculate distance matrix
  dist_mtx = np.full((n,n), np.inf)
  for g in graph:
    i = int(g[0]) - 1
    j = int(g[1]) - 1
    d = g[2]
    dist_mtx[i,j] = d
    dist_mtx[j,i] = d

  ## set diagonal to 0
  np.fill_diagonal(dist_mtx, 0)
 
  return dist_mtx


## print time costs
try:
  start = default_timer()
  dist_mtx = distance_matrix(np.array(data), nodes)
  stop = default_timer()
  print('Time: ', stop - start)
except:
  print('Time: inf')

Time:  0.00018018500000493987


##### Calculate Dijkstra distance

In [15]:
from timeit import default_timer
import numpy as np
# import timeout_decorator

# @timeout_decorator.timeout(100)
def dijkstra(source, distmatrix, n):
  dist = np.array([np.inf if i!=source else 0 for i in range(n)])
  Q = np.arange(n)
  while Q.shape[0]>0:
    u, Q = get_remove_min(Q, dist)
    U = get_neighbor(u, distmatrix, n)
    for v in U:
      newd = dist[u] + distmatrix[u,v]
      if newd < dist[v]:
        dist[v] = newd
  return dist


def get_remove_min(Q, dist):
  dmin = np.inf
  imin = -1
  for i in Q:
    if dist[i] < dmin:
      dmin = dist[i]
      imin = i
  Q = np.delete(Q, np.where(Q == imin))
  return imin, Q


def get_neighbor(u, d, n):
  neighbors = np.array([i for i in range(n) if d[i,u]!=np.inf and i!=u])
  return neighbors


def shortest_path(source, destination, distmatrix, n):
  dist = dijkstra(source, distmatrix, n)
  return dist[destination]


## print time costs
try:
  start = default_timer()
  dist = shortest_path(source, destination, dist_mtx, nodes)
  stop = default_timer()
  print('Time: ', stop - start)
  
except:
  print('Time: inf')
  raise

90.0
Time:  0.0010139059999119127
